In [1]:
%matplotlib inline

In [2]:
import fipy as fp

In [3]:
from fipy import numerix as nmx

In [26]:
mesh = fp.Gmsh2DIn3DSpace('''
    radius = 100.0;
    cellSize = 1.;
    
    // create inner 1/8 shell
    Point(1) = {0, 0, 0, cellSize};
    Point(2) = {-radius, 0, 0, cellSize};
    Point(3) = {0, radius, 0, cellSize};
    Point(4) = {0, 0, radius, cellSize};
    Circle(1) = {2, 1, 3};
    Circle(2) = {4, 1, 2};
    Circle(3) = {4, 1, 3};
    Line Loop(1) = {1, -3, 2} ;
    Ruled Surface(1) = {1};
    
    // create remaining 7/8 inner shells
    t1[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{1};}};
    t2[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{1};}};
    t3[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{1};}};
    t4[] = Rotate {{0,1,0},{0,0,0},-Pi/2} {Duplicata{Surface{1};}};
    t5[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{t4[0]};}};
    t6[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{t4[0]};}};
    t7[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{t4[0]};}};
    
    // create entire inner and outer shell
    Surface Loop(100)={1,t1[0],t2[0],t3[0],t7[0],t4[0],t5[0],t6[0]};
''', order=2).extrude(extrudeFunc=lambda r: 1.01 * r)

In [27]:
C = fp.CellVariable(mesh=mesh, name="C", hasOld=True)

In [28]:
Calpha = 0.05
Cbeta = 0.95
Cm = (Calpha + Cbeta) / 2.
A = 2.0
B = A / (Calpha - Cm)**2
D = Dalpha = Dbeta = 2 / (Cbeta - Calpha)
kappa = 2.0

In [29]:
f0 = -(A/2)*(C - Cm)**2 + (B/4)*(C - Cm)**4 + (Calpha/4)*(C - Calpha)**4 + (Cbeta/4)*(C - Cbeta)**4

In [30]:
f = f0 + (kappa/2.)*(C.grad.mag)**2
f.name = "f"

In [31]:
Cf = C.arithmeticFaceValue
eq = (fp.TransientTerm(coeff=1.)
      == fp.DiffusionTerm(coeff=D*(-A + 3*B*(Cf - Cm)**2 + 3*Calpha*(Cf - Calpha)**2 + 3*Cbeta*(Cf - Cbeta)**2))
      - fp.DiffusionTerm(coeff=(D,kappa)))

In [32]:
epsilon = 0.01
theta = nmx.arctan2(mesh.z, nmx.sqrt(mesh.x**2 + mesh.y**2))
phi = nmx.arctan2(mesh.y, mesh.x)
C.value = 0.45 + epsilon * nmx.cos(nmx.sqrt(233.) * theta) * nmx.sin(nmx.sqrt(239.) * phi)

In [11]:
fp.Viewer(vars=theta).plot()

viewer: NOT READY


In [12]:
fp.Viewer(vars=phi).plot()

viewer: NOT READY


In [76]:
viewer = fp.Viewer(vars=(C))

In [30]:
viewer.plot()

viewer: NOT READY


In [11]:
import os

In [33]:
direc = "1d-exp"

In [34]:
os.mkdir(direc)

In [14]:
os.chdir(os.path.join(os.getcwd(), direc))

In [36]:
!rm -r *

In [37]:
fp.tools.dump.write(mesh, filename="mesh.dmp")

In [38]:
fp.TSVViewer(vars=C).plot(filename="{step}.gz".format(step=0))

In [39]:
with open("stats.txt", 'w') as stats:
    stats.write("\t".join(["step", "t", "dt", "Cmin", "Cmax", "f"]) + "\n")

In [40]:
dexp = -5
elapsed = 0.
saved = elapsed
duration = 10000.
step = 0

In [41]:
while elapsed < duration:
    C.updateOld()
    dt = min(10, nmx.exp(dexp))
    elapsed += dt
    dexp += 0.01
    eq.solve(C, dt=dt)
    step += 1
    if elapsed - saved > 10:
        fp.TSVViewer(vars=C).plot(filename="{step}.gz".format(step=step))
        # viewer.plot()
        saved = elapsed
    with open("stats.txt", 'a') as stats:
        stats.write("\t".join([str(it) for it in [step, elapsed, dt, min(C), max(C), 
                                                  f.cellVolumeAverage * mesh.cellVolumes.sum()]]) + "\n")

/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:6: MaximumIterationWarning: Iterations: 1001. Relative error: 1.03729e-08
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:6: MaximumIterationWarning: Iterations: 1001. Relative error: 6.90972e-10
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:6: MaximumIterationWarning: Iterations: 1001. Relative error: 0.0607104
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:6: MaximumIterationWarning: Iterations: 1001. Relative error: 6.82997e-07
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:6: MaximumIterationWarning: Iterations: 1001. Relative error: 0.356507
/Users/guyer/anaconda/envs/fipy_petsc/lib/python2.7/site-packages/ipykernel/__main__.py:6: MaximumIterationWarning: Iterations: 1001. Relative error: 1.91353e-06
/Users/guyer/anaconda/envs/fipy_p

KeyboardInterrupt: 